In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import cv2
from collections import OrderedDict
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from matplotlib import pyplot as plt
from anomalib.pre_processing.pre_process import get_transforms, PreProcessor
import torchvision.models.detection as detection
from anomalib.data import InferenceDataset
from torchvision.ops import RoIAlign
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
import torchvision.models as models
import torch.nn.functional as F
from torch import nn, Tensor

from anomalib.models.rkde.feature_extractor import RegionExtractor as RegionExtractor2
from anomalib.models.rkde.region import RegionExtractor as RegionExtractor1
from anomalib.models.rkde.feature import FeatureExtractor as FeatureExtractor1
from anomalib.models.rkde.feature_extractor import FeatureExtractor as FeatureExtractor2

In [45]:
filename = "150.tif"
image = cv2.imread(filename)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations.
transforms = get_transforms(config="transforms.yaml")
pre_process = PreProcessor(config=transforms)

# Get the data via dataloader
dataset = InferenceDataset(path=filename, pre_process=pre_process)
dataloader = DataLoader(dataset)
i, data = next(enumerate(dataloader))

# Create the region extractor.
stage="rcnn"
use_original = False
region_extractor1 = RegionExtractor1(stage=stage, use_original=use_original).eval().cuda()
region_extractor2 = RegionExtractor2(stage=stage, use_original=use_original).eval().cuda()

# Forward-Pass the input
boxes1 = region_extractor1([image])
boxes2 = region_extractor2(data["image"].cuda())

# Feature Extractor
feature_extractor1 = FeatureExtractor1().eval().cuda()
feature_extractor2 = FeatureExtractor2().eval().cuda()
features1 = feature_extractor1(image, boxes1[0])
features2 = feature_extractor2(data["image"].cuda())